Here we focus on visualizing embeddings
- For a reference see:
    - [Tensorflow official guide](https://www.tensorflow.org/versions/r1.1/get_started/embedding_viz) 
    - Note the documentation is a bit outdated

In [1]:
# -----------------------
# - Libraries -
# -----------------------
# == Base libs ==
import os
import numpy as np
import pandas as pd  # we'll need this for saving data

# == TF libs ==
import tensorflow as tf
from tensorflow.contrib.tensorboard.plugins import projector

# == Keras Librarires ==
# - Data download -
from keras.datasets import mnist

/home/joshuah/venv3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/joshuah/venv3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/joshuah/venv3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/joshuah/venv3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/joshuah/venv3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
Using TensorFlow backend.


In [2]:
# -----------------------
# - Setting up data -
# -----------------------
path = os.path.join(os.getcwd(), 'mnist-tensorboard')

# --- Setting up directories ---
LOG_DIR = os.path.join(path,'log-1')
DATA_DIR = os.path.join(path, 'data')
METADATA_DIR = os.path.join(DATA_DIR,'metadata_labels')

A note on data storage for Keras:
 - Caches it's datasets in  <code> '~/.keras/datasets/' + path'</code> for future access
 - We'll load our data from DATA_DIR for reproducability
 - Notice that the format of the data as a as a <code>.npz</code>

In [3]:
# --- Loading data ---
print(  '...Loading data ...')
(x_train, y_train), (x_test, y_test) = mnist.load_data(path = 
                                                       os.path.join(DATA_DIR,'mnist.pz'))
print('Train samples shape: '+ str(x_train.shape))
print('Test samples shape: '+ str(x_test.shape))

...Loading data ...
Train samples shape: (60000, 28, 28)
Test samples shape: (10000, 28, 28)


In [4]:
# --- Reshaping data ----
test_shape = x_test.shape #(10000,28,28)
x_test = x_test.reshape(test_shape[0], test_shape[1]*test_shape[2] )

In [5]:
# --- Loading data for embedding ---
images = tf.Variable(x_test, name = 'images')

In [6]:
# --- Storing Label Metadata ----
import pandas as pd
y_test = pd.DataFrame(y_test)
y_test.to_csv(METADATA_DIR, index = False, header = False)

In [7]:
# -- Starting Tensorboard session ---
with tf.Session() as sess:
    saver = tf.train.Saver([images])

    sess.run(images.initializer)
    saver.save(sess, os.path.join(LOG_DIR, 'images.ckpt'))

    config = projector.ProjectorConfig()
    # One can add multiple embeddings.
    embedding = config.embeddings.add()
    embedding.tensor_name = images.name
    # Link this tensor to its metadata file (e.g. labels).
    embedding.metadata_path =  METADATA_DIR
    # Saves a config file that TensorBoard will read during startup.
    projector.visualize_embeddings(tf.summary.FileWriter(LOG_DIR), config)

# To run type: tensorboard --logdir=path/to/log-directory --port=6006
# Example tensorboard --logdir=./mnist-tensorboard/logs-1 --port=6006